In [644]:
import importlib
import utils
import transformers
import model
import losses

In [645]:
from model import *
from utils import *

In [646]:
importlib.reload(utils)
importlib.reload(transformers)
importlib.reload(model)
importlib.reload(losses)

<module 'losses' from 'd:\\Documents\\machinelearning\\cgm-paper\\src\\losses.py'>

In [647]:
print("Current working directory: ", os.getcwd())

Current working directory:  d:\Documents\machinelearning\cgm-paper\src


In [648]:
raw_dir = "../data/raw"
tuning_dir = "../data/tuning"

In [649]:
assert os.path.exists(raw_dir), "Raw data directory not found"
assert os.path.exists(tuning_dir), "Tuning data directory not found"

In [650]:
dataset = load_data(raw_dir)

Found 22 files in ../data/raw


train (139415, 47)
test (32912, 47)


In [651]:
val_dataset = load_data(tuning_dir)

Found 2 files in ../data/tuning
train (13630, 43)
test (3002, 43)


In [652]:
for key in dataset.keys():
    val_dataset[key] = reduce_classes(val_dataset[key])
    dataset[key] = reduce_classes(dataset[key])

In [653]:
dataset.get("train")['Time'].isnull().sum()

0

In [654]:
train_data = get_train_dataset(dataset['train'])
test_data = get_any_dataset(dataset['test'], dataset['train'])

KeyboardInterrupt: 

The tuning data will be merged with the training data and be fitted into the pipeline to be used for the validation data

In [ ]:
tuning_data = get_tuning_dataset(val_dataset['train'], dataset['train'])
val_data = get_any_dataset(val_dataset['test'], tuning_data)

In [ ]:
trainX, trainY = train_data.drop(["CGM"], axis = 1), train_data["CGM"]
trainX.shape, trainY.shape

((120010, 19), (120010,))

In [ ]:
testX, testY = test_data.drop(["CGM"], axis = 1), test_data["CGM"]
testX.shape, testY.shape

((26845, 19), (26845,))

In [ ]:
model = HybridModel()
model.fit(trainX, trainY, testX, testY, eval = True, tune = False)

Base: (18.6803674191468, 4.322079062111984, 0.08426079901940356)
Base + Residuals: (18.57569316090872, 4.309952802631221, 0.08355314473106679) Change: 4.602644486910961%
